<a href="https://colab.research.google.com/github/alex-jk/buyers-ID-manual-project/blob/main/process_studies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**This notebook extracts and cleans text from PDF reports for use in training the buyer profile extractor model.**

**Add input and output examples to jsonl file**

In [ ]:
!git clone https://github.com/alex-jk/buyers-ID-manual-project.git
%cd buyers-ID-manual-project

In [ ]:
import json

input_text = """
Buyers were willing to travel significant distances to reach underage victims.
One man cited the Atlanta airport as a regular meeting place.
"""

output_text = """
Buyers are often mobile and willing to travel, including to major hubs like the Atlanta airport.
"""

entry = {"input": input_text.strip(), "output": output_text.strip()}

with open("data/labeled_chunks.jsonl", "a", encoding="utf-8") as f:
    f.write(json.dumps(entry) + "\n")